In [1]:
import pandas as pd
import numpy as np

In [2]:
import pyarrow as pa
import pyarrow.parquet as pq

In [3]:
df = pd.read_csv('MaisTodos/california_housing_train.csv') #, iterator=True)

# Objetivo  


# 1 - Exploração:

1.1 - Qual a coluna com maior desvio padrão?

In [4]:
deviations = df[["housing_median_age","total_rooms","total_bedrooms","population","households","median_income", "median_house_value"]].std() 
deviations

housing_median_age        12.586937
total_rooms             2179.947071
total_bedrooms           421.499452
population              1147.852959
households               384.520841
median_income              1.908157
median_house_value    115983.764387
dtype: float64

In [5]:
f"coluna maior desvio padrão: {deviations.idxmax()}"

'coluna maior desvio padrão: median_house_value'

1.2 - Qual valor mínimo e o máximo?

In [6]:
min_max = pd.DataFrame()
min_max["min"] = df[["housing_median_age","total_rooms","total_bedrooms","population","households","median_income", "median_house_value"]].min()
min_max["max"]  = df[["housing_median_age","total_rooms","total_bedrooms","population","households","median_income", "median_house_value"]].max()

In [7]:
min_max

,min,max
housing_median_age,1.0000,52.0000
total_rooms,2.0000,37937.0000
total_bedrooms,1.0000,6445.0000
population,3.0000,35682.0000
households,1.0000,6082.0000
median_income,0.4999,15.0001
median_house_value,14999.0000,500001.0000


In [8]:
print(f"""valor minimo: {min_max["min"].min()}
valor maximo: {min_max["max"].max()}""")

valor minimo: 0.4999
valor maximo: 500001.0


## 2 - Trabalhando com colunas:

2.1 - Criar coluna hma_cat, baseada na coluna housing_median_age, conforme as regras abaixo:

    *  Se < 18 então de_0_ate_18.
    *  Se >= 18 E < 29 entao ate_29.
    *  Se >= 29 E < 37 entao ate_37.
    *  Se >= 37 então acima_37.  

In [9]:
df["hma_cat"] = df["housing_median_age"].mask(df["housing_median_age"] < 18, "de_0_ate_18") \
                                        .mask(df["housing_median_age"].between(18, 28), "ate_29")\
                                        .mask(df["housing_median_age"].between(29, 36), "ate_37")\
                                        .mask(df["housing_median_age"] >=37, "acima_37")

In [10]:
df["hma_cat"]

0        de_0_ate_18
1             ate_29
2        de_0_ate_18
3        de_0_ate_18
4             ate_29
            ...     
16995       acima_37
16996         ate_37
16997    de_0_ate_18
16998         ate_29
16999       acima_37
Name: hma_cat, Length: 17000, dtype: object

In [11]:
df

,longitude,latitude,housing_median_age,total_rooms,total_bedrooms,population,households,median_income,median_house_value,hma_cat
0,-114.31,34.19,15.0,5612.0,1283.0,1015.0,472.0,1.4936,66900.0,de_0_ate_18
1,-114.47,34.40,19.0,7650.0,1901.0,1129.0,463.0,1.8200,80100.0,ate_29
2,-114.56,33.69,17.0,720.0,174.0,333.0,117.0,1.6509,85700.0,de_0_ate_18
3,-114.57,33.64,14.0,1501.0,337.0,515.0,226.0,3.1917,73400.0,de_0_ate_18
4,-114.57,33.57,20.0,1454.0,326.0,624.0,262.0,1.9250,65500.0,ate_29
...,...,...,...,...,...,...,...,...,...,...
16995,-124.26,40.58,52.0,2217.0,394.0,907.0,369.0,2.3571,111400.0,acima_37
16996,-124.27,40.69,36.0,2349.0,528.0,1194.0,465.0,2.5179,79000.0,ate_37
16997,-124.30,41.84,17.0,2677.0,531.0,1244.0,456.0,3.0313,103600.0,de_0_ate_18
16998,-124.30,41.80,19.0,2672.0,552.0,1298.0,478.0,1.9797,85800.0,ate_29


2.2 - Criar a coluna c_ns:

    * Onde longitude abaixo (<) de -119 recebe o valor norte e acima(>=) sul. 

In [12]:
df["c_ns"]= np.where(df["longitude"]< -119, "norte", "sul")

In [13]:
df

,longitude,latitude,housing_median_age,total_rooms,total_bedrooms,population,households,median_income,median_house_value,hma_cat,c_ns
0,-114.31,34.19,15.0,5612.0,1283.0,1015.0,472.0,1.4936,66900.0,de_0_ate_18,sul
1,-114.47,34.40,19.0,7650.0,1901.0,1129.0,463.0,1.8200,80100.0,ate_29,sul
2,-114.56,33.69,17.0,720.0,174.0,333.0,117.0,1.6509,85700.0,de_0_ate_18,sul
3,-114.57,33.64,14.0,1501.0,337.0,515.0,226.0,3.1917,73400.0,de_0_ate_18,sul
4,-114.57,33.57,20.0,1454.0,326.0,624.0,262.0,1.9250,65500.0,ate_29,sul
...,...,...,...,...,...,...,...,...,...,...,...
16995,-124.26,40.58,52.0,2217.0,394.0,907.0,369.0,2.3571,111400.0,acima_37,norte
16996,-124.27,40.69,36.0,2349.0,528.0,1194.0,465.0,2.5179,79000.0,ate_37,norte
16997,-124.30,41.84,17.0,2677.0,531.0,1244.0,456.0,3.0313,103600.0,de_0_ate_18,norte
16998,-124.30,41.80,19.0,2672.0,552.0,1298.0,478.0,1.9797,85800.0,ate_29,norte


2.3 - Renomer as colunas:

    * hma_cat > age
    * c_ns > california_region

In [14]:
df = df.rename(columns={"hma_cat": "age", "c_ns": "california_region"})

In [15]:
df

,longitude,latitude,housing_median_age,total_rooms,total_bedrooms,population,households,median_income,median_house_value,age,california_region
0,-114.31,34.19,15.0,5612.0,1283.0,1015.0,472.0,1.4936,66900.0,de_0_ate_18,sul
1,-114.47,34.40,19.0,7650.0,1901.0,1129.0,463.0,1.8200,80100.0,ate_29,sul
2,-114.56,33.69,17.0,720.0,174.0,333.0,117.0,1.6509,85700.0,de_0_ate_18,sul
3,-114.57,33.64,14.0,1501.0,337.0,515.0,226.0,3.1917,73400.0,de_0_ate_18,sul
4,-114.57,33.57,20.0,1454.0,326.0,624.0,262.0,1.9250,65500.0,ate_29,sul
...,...,...,...,...,...,...,...,...,...,...,...
16995,-124.26,40.58,52.0,2217.0,394.0,907.0,369.0,2.3571,111400.0,acima_37,norte
16996,-124.27,40.69,36.0,2349.0,528.0,1194.0,465.0,2.5179,79000.0,ate_37,norte
16997,-124.30,41.84,17.0,2677.0,531.0,1244.0,456.0,3.0313,103600.0,de_0_ate_18,norte
16998,-124.30,41.80,19.0,2672.0,552.0,1298.0,478.0,1.9797,85800.0,ate_29,norte


### Escrevendo o resultado localmente em parquet, armazenar os dados no seguinte formato:


| Coluna              | Datatype    |
| --------------------| ----------- |
| `age`               | `string`    |
| `california_region` | `string`    |
| `total_rooms`       | `double`    |
| `total_bedrooms`    | `double`    |
| `population`        | `double`    |
| `households`        | `double`    |
| `median_house_value`| `double`    |


In [16]:
columns = ["age","california_region","total_rooms","total_bedrooms","population","households","median_house_value"]

In [17]:
check_table = pa.Table.from_pandas(df[columns], preserve_index=True)
pq.write_table(check_table, 'check.parquet')

## 3 - Agregações:

3.1 - Escreva um arquivo no formato Parquet localmente considerando o dataframe final, crie a seguinte analise:

    * Age
    * California_region
    * S_population: Soma de population
    * M_median_house_value: Média de median_house_value

In [18]:
final_df_group = df.groupby(["california_region", "age"])

In [19]:
S_population = final_df_group['population'].sum().rename("S_population")
M_median_house_value =  final_df_group['median_house_value'].mean().rename("M_median_house_value")

In [20]:
final_df = pd.merge(S_population, M_median_house_value, right_index=True, left_index=True).reset_index()

In [21]:
final_df.sort_values("M_median_house_value", ascending=False)

,california_region,age,S_population,M_median_house_value
4,sul,acima_37,2519076.0,227694.827766
5,sul,ate_29,3905630.0,220571.658462
0,norte,acima_37,2114160.0,217732.956241
6,sul,ate_37,3435282.0,212266.993286
7,sul,de_0_ate_18,4157987.0,209407.565043
2,norte,ate_37,1792950.0,195766.603215
1,norte,ate_29,3107411.0,188724.023810
3,norte,de_0_ate_18,3270261.0,177826.697686


In [22]:
final_table = pa.Table.from_pandas(final_df, preserve_index=True)
pq.write_table(final_table, 'final.parquet')